# Uber Data Optimization

The purpose of the research would be to build an application to optimize Uber drivers’ rides in Boston. With this application Uber drivers should have a higher driving frequency, leading to 
higher profit margins in less time. Factors and assumptions to take into consideration include but are not limited to customer pickup frequency, time of day, weather, and popular events happening in the city. Through this project, we aim to learn and apply predictive models like time series analysis, as well as machine learning methods such as Support Vector Machines, K- Nearest Neighbors, and Naïve Bayes. Uber drivers only have a finite amount time for driving, so why not make the most out of their driving time? 
Here, we focus on the Uber ride in Boston and Cambridge areas.
 


Background -
Being an Uber driver may sound lucrative, but how real is it and can the time utilization be maximized as they also need to pay for expenses for running the car or its insurance. An Uber driver may happen to be driving at a particular location, but there is no guarantee that there are passengers there, if any. Logically speaking, it is better to have accurate predictions for an Uber driver to decide upon his/her own schedule, maximize earnings in a shorter time, and on top of that knowing all the best driving locations where ride-hailing customers are densest. Conducting data analysis, training predictive models, and building an application would come in handy for the drivers, all in which are the aim of the project. 

This notebook deals with obtaining responses from Uber API and storing them into SQLite database for further anallysis.


Dataset sources-  
Uber api -https://developer.uber.com/
Uber Rides Python SDK (beta) -https://github.com/uber/rides-python-sdk
Yelp data on uber rides in boston - https://www.yelp.com/search?find_desc=uber&find_loc=Boston%2C+MA



First Step - To query yelp api to get location latitude and longitude

In [106]:
from __future__ import print_function
import json
import pandas as pd

Storing yelp data in sqllite database

In [107]:
import sqlite3
yelp_db_nw = 'yelp_db_nw.sqlite' 
conn = sqlite3.connect(yelp_db_nw) 
c = conn.cursor()

In [108]:
for row in conn.execute("pragma table_info('yelp_reviews')").fetchall(): #Checking table contents
    print (row)

(0, 'yelp_id', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'image_url', 'TEXT', 0, None, 0)
(4, 'url', 'TEXT', 0, None, 0)
(5, 'phone', 'INTEGER', 0, None, 0)
(6, 'display_phone', 'TEXT', 0, None, 0)
(7, 'review_count', 'INTEGER', 0, None, 0)
(8, 'categories__alias', 'TEXT', 0, None, 0)
(9, 'categories__title', 'TEXT', 0, None, 0)
(10, 'rating', 'REAL', 0, None, 0)
(11, 'location__address1', 'TEXT', 0, None, 0)
(12, 'location__address3', 'TEXT', 0, None, 0)
(13, 'location__city', 'TEXT', 0, None, 0)
(14, 'location__zip_code', 'INTEGER', 0, None, 0)
(15, 'location__country', 'TEXT', 0, None, 0)
(16, 'location__state', 'TEXT', 0, None, 0)
(17, 'location__display_address', 'TEXT', 0, None, 0)
(18, 'location__cross_streets', 'TEXT', 0, None, 0)
(19, 'coordinates__latitude', 'REAL', 0, None, 0)
(20, 'coordinates__longitude', 'REAL', 0, None, 0)
(21, 'photos', 'TEXT', 0, None, 0)
(22, 'Unnamed: 22', 'REAL', 0, None, 0)
(23, 'Unnamed: 23', 'REAL', 

In [109]:
data_nwf = pd.read_csv("yelp_data.csv", encoding ='latin-1') #reading yelp csv
data_nwf.head()

,yelp_id,id,name,image_url,url,phone,display_phone,review_count,categories__alias,categories__title,...,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,1,coolidge-corner-theatre-brookline-2,Coolidge Corner Theatre,https://s3-media3.fl.yelpcdn.com/bphoto/RFVooG...,https://www.yelp.com/biz/coolidge-corner-theat...,16177342500,(617) 734-2500,357,movietheaters,Cinema,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,showcase-superlux-newton-2,Showcase SuperLux,https://s3-media2.fl.yelpcdn.com/bphoto/GOtzKl...,https://www.yelp.com/biz/showcase-superlux-new...,16176585160,(617) 658-5160,254,movietheaters,Cinema,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,kendall-square-cinema-cambridge,Kendall Square Cinema,https://s3-media2.fl.yelpcdn.com/bphoto/GOtzKl...,https://www.yelp.com/biz/kendall-square-cinema...,16176211202,(617) 621-1202,274,movietheaters,Cinema,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,the-lyric-stage-company-of-boston-boston,The Lyric Stage Company of Boston,https://s3-media1.fl.yelpcdn.com/bphoto/C8wst5...,https://www.yelp.com/biz/the-lyric-stage-compa...,16175855678,(617) 585-5678,22,theater,Performing Arts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,dedham-community-theatre-dedham,Dedham Community Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/0u_oQu...,https://www.yelp.com/biz/dedham-community-thea...,17813261463,(781) 326-1463,35,movietheaters,Cinema,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now use the latitude and longitude obtained from yelp api to query uber api
Also, the minimum distance defined for a profitable ride is 1 mile. This is a straight line distance calculated through Geopy API.

Now hit Uber API...

In [110]:

from uber_rides.session import Session
from uber_rides.client import UberRidesClient
import pandas as pd
import json
from datetime import datetime


session = Session(server_token='Uvu3eEPnLtPKCbTU7KrCko5jo1ua4CVgYAqd0JfO')
client = UberRidesClient(session)

In [111]:
def find_first(): #function to query to select random first location
    df = pd.read_sql('SELECT yelp_id, coordinates__latitude, coordinates__longitude FROM yelp_reviews where yelp_id IN (SELECT yelp_id FROM yelp_reviews ORDER BY RANDOM() LIMIT 1)', con=conn)
    df.head()
    return df

In [112]:
def find_second(): #function to query to select randon second location
    df_2 = pd.read_sql('SELECT yelp_id, coordinates__latitude, coordinates__longitude FROM yelp_reviews where yelp_id IN (SELECT yelp_id FROM yelp_reviews ORDER BY RANDOM() LIMIT 1)', con = conn)
    df_2.head()
    return df_2

In [113]:
find_first() # first query result

,yelp_id,coordinates__latitude,coordinates__longitude
0,15,42.344343,-71.098794


In [114]:
find_second() #second query result

,yelp_id,coordinates__latitude,coordinates__longitude
0,26,42.331336,-71.095474


In [115]:
import datetime  #first to define date time for east zone which is the area of interest
import json
import os

class EST5EDT(datetime.tzinfo):

    def utcoffset(self, dt):
        return datetime.timedelta(hours=-5) + self.dst(dt)

    def dst(self, dt):
        d = datetime.datetime(dt.year, 3, 8)        #2nd Sunday in March
        self.dston = d + datetime.timedelta(days=6-d.weekday())
        d = datetime.datetime(dt.year, 11, 1)       #1st Sunday in Nov
        self.dstoff = d + datetime.timedelta(days=6-d.weekday())
        if self.dston <= dt.replace(tzinfo=None) < self.dstoff:
            return datetime.timedelta(hours=1)
        else:
            return datetime.timedelta(0)

    def tzname(self, dt):
        return 'EST5EDT'


In [116]:
def actual_second(): #function to hit Uber API and store the result in a JSON file for further analysis
    df_1 = find_first()
    df_2 = find_second()
    from geopy.distance import vincenty
    start_loc = (df_1['coordinates__latitude'][0], df_1['coordinates__longitude'][0])
  
    end_loc = (df_2['coordinates__latitude'][0], df_2['coordinates__longitude'][0])
   
    distance = vincenty(start_loc, end_loc).miles
    if(distance > 1):
        
        response = client.get_price_estimates(
        start_latitude= df_1['coordinates__latitude'][0],
        start_longitude= df_1['coordinates__longitude'][0],
        end_latitude=  df_2['coordinates__latitude'][0],
        end_longitude= df_2['coordinates__longitude'][0],
        seat_count=2
        )
        prices = response.json.get("prices")
       
        dt = datetime.datetime.now(tz=EST5EDT()) 
        
        for price in prices:
            price["time"]= dt.strftime('%H:%M:%S')
            price["Date-time"] = dt.strftime('%Y-%m-%d %H:%M:%S')
            price["start_latitude"] = df_1['coordinates__latitude'][0]
            price["start_longitude"] = df_1['coordinates__longitude'][0]
            price["end_latitude"] = df_2['coordinates__latitude'][0]
            price["end_longitude"] = df_2['coordinates__longitude'][0]
        
        df = pd.DataFrame(prices)
        df = df.append(df)
        out = df.to_json(orient='records')
        with open('uber_data.json', 'w+') as outfile:
            json.dump(out, outfile)
            outfile.close()
    else:
        actual_second() 
        
        
        
        
    


In [117]:
actual_second() # Call to above function

A Cron job was set up by loading the uber API query script on the server to collect real time data. The result were appended to a json file and used for further analysis

In [118]:
j = json.load(open('uber_13_12_17.json')) # Read the files retrived from server
dj = pd.read_json(j)
df_uber = pd.DataFrame(dj)


In [119]:
import json, os
def load_uber(j):
    p=os.path.join("", j)
    print (p)
    with open(p, 'rU') as f:
      data = [json.loads(row) for row in f]
    return data

In [120]:
# import sqlite3
# Will create uber_db.sqlite if it doesn't exist.
uber_db = 'uber_db.sqlite' 
conn = sqlite3.connect(uber_db) 
c = conn.cursor()

In [121]:
df_uber.to_sql('uber_data',              # Name of the table.
             con=conn,                    # The handle to the file that is set up.
             if_exists='replace',         # Overwrite, append, or fail.
             index=False) 

In [122]:


for row in connu.execute("pragma table_info('uber_data')").fetchall(): # Check contents of the table
    print (row)


(0, 'Date-time', 'TEXT', 0, None, 0)
(1, 'currency_code', 'TEXT', 0, None, 0)
(2, 'display_name', 'TEXT', 0, None, 0)
(3, 'distance', 'REAL', 0, None, 0)
(4, 'duration', 'INTEGER', 0, None, 0)
(5, 'end_latitude', 'REAL', 0, None, 0)
(6, 'end_longitude', 'REAL', 0, None, 0)
(7, 'estimate', 'TEXT', 0, None, 0)
(8, 'high_estimate', 'REAL', 0, None, 0)
(9, 'localized_display_name', 'TEXT', 0, None, 0)
(10, 'low_estimate', 'REAL', 0, None, 0)
(11, 'product_id', 'TEXT', 0, None, 0)
(12, 'start_latitude', 'REAL', 0, None, 0)
(13, 'start_longitude', 'REAL', 0, None, 0)
(14, 'time', 'TEXT', 0, None, 0)


In [123]:
df = pd.read_sql('SELECT * FROM uber_data', con=conn) #read uber table
df.head()

,Date-time,currency_code,display_name,distance,duration,end_latitude,end_longitude,estimate,high_estimate,localized_display_name,low_estimate,product_id,start_latitude,start_longitude,time
0,2017-12-05 22:47:56,USD,uberPOOL,8.24,1080,42.314857,-71.056782,$8-11,12.0,uberPOOL,8.0,997acbb5-e102-41e1-b155-9df7de0a73f2,42.342588,-71.122463,22:47:56
1,2017-12-05 22:47:56,USD,uberX,8.24,1080,42.314857,-71.056782,$18-23,23.0,uberX,18.0,55c66225-fbe7-4fd5-9072-eab1ece5e23e,42.342588,-71.122463,22:47:56
2,2017-12-05 22:47:56,USD,uberSUV,8.24,1080,42.314857,-71.056782,$50-63,63.0,uberSUV,50.0,6d318bcc-22a3-4af6-bddd-b409bfce1546,42.342588,-71.122463,22:47:56
3,2017-12-05 22:47:56,USD,uberXL,8.24,1080,42.314857,-71.056782,$28-35,35.0,uberXL,28.0,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,42.342588,-71.122463,22:47:56
4,2017-12-05 22:47:56,USD,UberBLACK,8.24,1080,42.314857,-71.056782,$41-51,51.0,UberBLACK,41.0,6c84fd89-3f11-4782-9b50-97c468b19529,42.342588,-71.122463,22:47:56


In [124]:
data_nwf.to_sql('yelp_reviews',             # Name of the table.
             con=conn,                    # The handle to the file that is set up.
             if_exists='replace',         # Overwrite, append, or fail.
             index=False)                 # Add index as column.

C:\Users\amaha\Anaconda3\envs\py35\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [125]:
df_y = pd.read_sql('SELECT * FROM yelp_reviews', con =conn) # Check yelp review table
df_y.head()

,yelp_id,id,name,image_url,url,phone,display_phone,review_count,categories__alias,categories__title,...,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
0,1,coolidge-corner-theatre-brookline-2,Coolidge Corner Theatre,https://s3-media3.fl.yelpcdn.com/bphoto/RFVooG...,https://www.yelp.com/biz/coolidge-corner-theat...,16177342500,(617) 734-2500,357,movietheaters,Cinema,...,None,None,None,None,None,None,None,None,None,None
1,2,showcase-superlux-newton-2,Showcase SuperLux,https://s3-media2.fl.yelpcdn.com/bphoto/GOtzKl...,https://www.yelp.com/biz/showcase-superlux-new...,16176585160,(617) 658-5160,254,movietheaters,Cinema,...,None,None,None,None,None,None,None,None,None,None
2,3,kendall-square-cinema-cambridge,Kendall Square Cinema,https://s3-media2.fl.yelpcdn.com/bphoto/GOtzKl...,https://www.yelp.com/biz/kendall-square-cinema...,16176211202,(617) 621-1202,274,movietheaters,Cinema,...,None,None,None,None,None,None,None,None,None,None
3,4,the-lyric-stage-company-of-boston-boston,The Lyric Stage Company of Boston,https://s3-media1.fl.yelpcdn.com/bphoto/C8wst5...,https://www.yelp.com/biz/the-lyric-stage-compa...,16175855678,(617) 585-5678,22,theater,Performing Arts,...,None,None,None,None,None,None,None,None,None,None
4,5,dedham-community-theatre-dedham,Dedham Community Theatre,https://s3-media2.fl.yelpcdn.com/bphoto/0u_oQu...,https://www.yelp.com/biz/dedham-community-thea...,17813261463,(781) 326-1463,35,movietheaters,Cinema,...,None,None,None,None,None,None,None,None,None,None


Use joins to query and analyses result of both Uber and Yelp table. A joion example to find price estimates, uber ride type and category of the place of visit is illustrated below:

In [126]:
df_j = pd.read_sql('SELECT uber_data.display_name, uber_data.product_id, uber_data.estimate, yelp_reviews.location__address1, yelp_reviews.categories__title FROM uber_data INNER JOIN yelp_reviews ON uber_data.end_latitude=yelp_reviews.coordinates__latitude', con =conn)

In [127]:
df_j.head()

,display_name,product_id,estimate,location__address1,categories__title
0,uberPOOL,997acbb5-e102-41e1-b155-9df7de0a73f2,$6-9,1509 Blue Hill Ave,Hot Dogs
1,uberX,55c66225-fbe7-4fd5-9072-eab1ece5e23e,$15-19,1509 Blue Hill Ave,Hot Dogs
2,uberSUV,6d318bcc-22a3-4af6-bddd-b409bfce1546,$43-54,1509 Blue Hill Ave,Hot Dogs
3,uberXL,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,$23-30,1509 Blue Hill Ave,Hot Dogs
4,UberBLACK,6c84fd89-3f11-4782-9b50-97c468b19529,$34-43,1509 Blue Hill Ave,Hot Dogs


In [128]:
df_j.tail()

,display_name,product_id,estimate,location__address1,categories__title
45257,uberSUV,6d318bcc-22a3-4af6-bddd-b409bfce1546,$48-59,33 Union St,Sandwiches
45258,uberXL,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,$26-33,33 Union St,Sandwiches
45259,UberBLACK,6c84fd89-3f11-4782-9b50-97c468b19529,$38-48,33 Union St,Sandwiches
45260,uberWAV,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,$17-21,33 Union St,Sandwiches
45261,TAXI,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Metered,33 Union St,Sandwiches
